In [ ]:
using NeuralPDE, Optimization, OptimizationOptimJL, Roots,
      LineSearches
using ModelingToolkit, IntervalSets 
using IntervalSets
using Plots, Printf
using Lux, LuxCUDA, ComponentArrays, Random
using JLD2, LinearAlgebra 

In [ ]:
@parameters t, x, y
@variables B1(..), B2(..)
Dx = Differential(x)
Dy = Differential(y)
Dt = Differential(t)



In [ ]:
#@load "toy_MHD_2D.jld2" config domains eqs bcs depvars loss res.u 
@load "toy_MHD_2D.jld2" config domains eqs bcs loss resu


In [ ]:
# Neural network
input_ = length(domains)
n = config[:N_neurons]
#chain = [Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, 1)) for _ in 1:2] #this work without GPU
#chain = Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, 2))
chain = Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, n, σ), Dense(n, 2))

#strategy = QuadratureTraining()
strategy = QuasiRandomTraining(100)
#strategy = GridTraining(config[:dx])
#ps = Lux.setup(Random.default_rng(), chain)[1] 
#ps = ps |> ComponentArray |> gpud
discretization = PhysicsInformedNN(chain, strategy) #; init_params = ps)
phi_d = discretization.phi

In [ ]:
plot(log10.(loss))

In [ ]:
xs, ys = [0.0:0.01:1.0 for d in 1:2]
t_p = config[:tmax]
u_predict = [[Array(phi_d([t_p, x, y], resu))[i] for x in xs for y in ys] for i in 1:2]

ps = []
for i in 1:2
    p1 = plot(xs, ys, u_predict[:,:][i], 
        linetype = :contourf, 
        #st = :surface,
        title = "predict B$i", aspect_ratio = 1)
    push!(ps, p1)
end

plot(ps[1], ps[2], layout = (1,2))

In [ ]:
function plot_(phi_d, resu)
    # Animate
    anim = @animate for (i, t_p) in enumerate(0:0.05:config[:tmax])
        @info "Animating frame $i..."
        B1_approx = [Array(phi_d([t_p, x, y], resu))[1] for x in xs for y in ys]
        B2_approx = [Array(phi_d([t_p, x, y], resu))[2] for x in xs for y in ys]
        title = @sprintf("B1, t = %.3f", t_p)
        p1 = plot(xs, ys, B1_approx, st = :surface, label = "B1", title = title)
        title = @sprintf("B2, t = %.3f", t_p)
        p2 = plot(xs, ys, B2_approx, st = :surface, label = "B2", title = title)
        plot(p1, p2)
    end
    gif(anim, "Toy_MHD_pde.gif", fps = 10)
end

plot_(phi_d, resu)